In [1]:
from google.colab import files
uploaded = files.upload()

Saving 격자:동별 인구수.csv to 격자:동별 인구수.csv


격자/동별 인구수.csv ->
1. qgis를 활용하여 격자별 인구 통계의 각 폴리곤 내의 중심점을 구함
2. 중심점의 위도, 경도 값을 추가
3. 양천구 행정동 경계 파일을 병합하여 동 정보 추가

In [14]:
import pandas as pd

# 파일 이름을 적절하게 수정
file_name = '/content/격자:동별 인구수.csv'

# CSV 파일을 Pandas 데이터프레임으로 읽기
df = pd.read_csv(file_name)

print(df)

       val EMD_KOR_NM_max  xcoord   ycoord
0      NaN            NaN  940125  1945875
1      NaN            NaN  940125  1946375
2      NaN            NaN  940125  1946625
3      NaN            NaN  940125  1948375
4    164.0            신월동  940125  1948625
..     ...            ...     ...      ...
341    NaN             목동  945875  1948875
342    NaN            NaN  945875  1949125
343    NaN            NaN  946125  1948125
344    NaN            NaN  946125  1948375
345    NaN            NaN  946125  1948625

[346 rows x 4 columns]


In [55]:
print(df)

     xcoord   ycoord  법정동  격자별 인구수  동별 반려동물수  동별 인구수      반려동물수
0    940125  1945875  NaN        0         0       0   0.000000
1    940125  1946375  NaN        0         0       0   0.000000
2    940125  1946625  NaN        0         0       0   0.000000
3    940125  1948375  NaN        0         0       0   0.000000
4    940125  1948625  신월동      164      7375  116015  10.425376
..      ...      ...  ...      ...       ...     ...        ...
341  945875  1948875   목동        0      8072  144260   0.000000
342  945875  1949125  NaN        0         0       0   0.000000
343  946125  1948125  NaN        0         0       0   0.000000
344  946125  1948375  NaN        0         0       0   0.000000
345  946125  1948625  NaN        0         0       0   0.000000

[346 rows x 7 columns]


In [16]:
df['법정동'] = df.iloc[:, 1]

In [17]:
df['인구수'] = df.iloc[:, 0]

In [18]:
df = df.drop(df.columns[[0, 1]], axis=1)

In [22]:
# '인구수' 열의 값 정수로 변환 및 NaN을 0으로 대체
df['인구수'].fillna(0, inplace=True)
df['인구수'] = df['인구수'].astype(int)

In [25]:
total_sum = df['인구수'].sum()
print(total_sum)

430780


In [30]:
# petNum, petSum 열을 추가하고 기본값으로 0 할당
df['petNum'] = 0
df['petSum'] = 0

In [33]:
df.loc[df['법정동'] == '신월동', 'petSum'] = 7375
df.loc[df['법정동'] == '목동', 'petSum'] = 8072
df.loc[df['법정동'] == '신정동', 'petSum'] = 9116

In [34]:
df['peopleSum'] = 0

In [39]:
temp_sum = df[df['법정동'] == '목동']['인구수'].sum()
df.loc[df['법정동'] == '목동', 'peopleSum'] = temp_sum

In [41]:
temp_sum = df[df['법정동'] == '신정동']['인구수'].sum()
df.loc[df['법정동'] == '신정동', 'peopleSum'] = temp_sum

In [42]:
temp_sum = df[df['법정동'] == '신월동']['인구수'].sum()
df.loc[df['법정동'] == '신월동', 'peopleSum'] = temp_sum

In [43]:
df['petNum'] = (df['petSum'] * df['인구수']) / df['peopleSum']

In [47]:
# 결과 잘 나왔는지 확인
result = df[df['법정동'] == '신정동']
print(result)

     xcoord   ycoord  법정동   인구수      petNum  petSum  peopleSum
11   940375  1946375  신정동   642   35.534135    9116     164700
26   940625  1945875  신정동   524   29.002939    9116     164700
27   940625  1946125  신정동   544   30.109921    9116     164700
28   940625  1946375  신정동   849   46.991403    9116     164700
47   940875  1945875  신정동  1456   80.588318    9116     164700
..      ...      ...  ...   ...         ...     ...        ...
284  944875  1946375  신정동  2997  165.881311    9116     164700
285  944875  1946625  신정동  3126  173.021348    9116     164700
286  944875  1946875  신정동  3317  183.593030    9116     164700
287  944875  1947125  신정동  2902  160.623145    9116     164700
303  945125  1947125  신정동  1159   64.149630    9116     164700

[118 rows x 7 columns]


In [48]:
petnumsum = df['petNum'].sum()
print(petnumsum)

24563.0


In [49]:
df['petNum'].fillna(0, inplace=True)

In [51]:
df['반려동물수'] = df.iloc[:, 4]
df = df.drop(df.columns[[4]], axis=1)

In [53]:
df = df.rename(columns={'인구수': '격자별 인구수', 'petSum': '동별 반려동물수', 'peopleSum': '동별 인구수'})

In [54]:
df.to_csv('격자별_반려동물수_추정량.csv', index=False)